In [1]:
# Allows code to live reload
%load_ext autoreload
%autoreload 2

## 1. Install
```sh
# Install prerequisites
pip install pyodc

```

## 2. Make an ECMWF account
- Go to ecmwf.int/, click login at the top right and click register to make a new account.
- Once logged in, go to api.ecmwf.int/v1/key/ to get your key. 
- Put it in `~/.ecmwfapirc` as directed.

In [2]:
# Load in the ECMWF token 
from pathlib import Path
import json
import requests
from IPython.display import JSON
from datetime import datetime as dt
from datetime import timedelta, timezone
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

with open(Path("~/.ecmwfapirc").expanduser(), "r") as f:
    api_creds = json.load(f)

print("Checking API credentials")
r = requests.get(f"https://api.ecmwf.int/v1/who-am-i?token={api_creds['key']}")
if r.status_code == 403: print("Your credentials are either wrong or need to be renewed at https://api.ecmwf.int/v1/key/")
r.raise_for_status()
JSON(r.json())

Checking API credentials


<IPython.core.display.JSON object>

In [3]:
session = requests.Session()
session.headers["Authorization"] = f"Bearer {api_creds['key']}"

In [4]:
url = "http://ionbeam-ichange.ecmwf-ichange.f.ewcloud.host/api/v1/"
# url = "http://localhost:5002/api/v1/"

In [5]:
from datetime import datetime

stations = session.get(url + "stations").json()
print(f"{len(stations) = }")

len(stations) = 78


In [6]:
granules = session.get(url + "list").json()
print(f"{len(granules) = }")

len(granules) = 78


In [7]:
from collections import Counter
types = Counter(s["platform"] for s in stations)
types

Counter({'acronet': 50, 'meteotracker': 28})

In [8]:
from collections import defaultdict
by_platform = defaultdict(list)
for s in stations:
    by_platform[s["platform"]].append(s)

print("Most recently updated entry from each platform\n")
for platform, platform_stations in by_platform.items():
    print(platform)
    print(json.dumps(
        sorted(platform_stations, key = lambda s : datetime.fromisoformat(s["time_span"][1]) )[-1],
        indent = 4,
        ))

Most recently updated entry from each platform

meteotracker
{
    "name": "MeteoTracker Track",
    "description": "A MeteoTracker Track.",
    "platform": "meteotracker",
    "external_id": "67573d05df36910918a5f274",
    "internal_id": "0a6f87f7c6170322",
    "location": [
        8.916138900000002,
        44.48129175
    ],
    "time_span": [
        "2024-12-09T18:55:01Z",
        "2024-12-09T19:08:28Z"
    ],
    "authors": [
        {
            "name": "genova_living_lab_1"
        },
        {
            "name": "meteotracker"
        }
    ],
    "mars_request": {
        "class": "rd",
        "expver": "xxxx",
        "stream": "lwda",
        "aggregation_type": "tracked",
        "date": "20241209",
        "platform": "meteotracker",
        "internal_id": "0a6f87f7c6170322"
    }
}
acronet
{
    "name": "Arenella",
    "description": "An Acronet station",
    "platform": "acronet",
    "external_id": "arenella",
    "internal_id": "a7d22f057bb346b9",
    "location": 

## Obtain a whole meteotracker track

For meteotracker tracks it is sufficient to simply use the "mars_request" as a key to the retrieve endpoint (along with format=csv) to download the data.

In [9]:
from io import BytesIO

example_station = by_platform["meteotracker"][-1]

args = {
    "format" : "csv"
}

data = session.get(url + "retrieve", params = example_station["mars_request"] | args)

df = pd.read_csv(BytesIO(data.content))
df

class expver stream   project      platform aggregation_type  \
0       rd   xxxx   lwda  I-CHANGE  meteotracker          tracked   
1       rd   xxxx   lwda  I-CHANGE  meteotracker          tracked   
2       rd   xxxx   lwda  I-CHANGE  meteotracker          tracked   
3       rd   xxxx   lwda  I-CHANGE  meteotracker          tracked   
4       rd   xxxx   lwda  I-CHANGE  meteotracker          tracked   
...    ...    ...    ...       ...           ...              ...   
1346    rd   xxxx   lwda  I-CHANGE  meteotracker          tracked   
1347    rd   xxxx   lwda  I-CHANGE  meteotracker          tracked   
1348    rd   xxxx   lwda  I-CHANGE  meteotracker          tracked   
1349    rd   xxxx   lwda  I-CHANGE  meteotracker          tracked   
1350    rd   xxxx   lwda  I-CHANGE  meteotracker          tracked   

              source_name               external_id       internal_id  \
0     Gert-Jan Steeneveld  67569684df36910918441085  f620b1ae408d1184   
1     Gert-Jan Steeneveld  67569684df36910918441085  f620b1ae408d1184   
2     Gert-Jan Steeneveld  67569684df36910918441085  f620b1ae408d1184   
3     Gert-Jan Steeneveld  67569684df36910918441085  f620b1ae408d1184   
4     Gert-Jan Steeneveld  67569684df36910918441085  f620b1ae408d1184   
...                   ...                       ...               ...   
1346  Gert-Jan Steeneveld  67569684df36910918441085  f620b1ae408d1184   
1347  Gert-Jan Steeneveld  67569684df36910918441085  f620b1ae408d1184   
1348  Gert-Jan Steeneveld  67569684df36910918441085  f620b1ae408d1184   
1349  Gert-Jan Steeneveld  67569684df36910918441085  f620b1ae408d1184   
1350  Gert-Jan Steeneveld  67569684df36910918441085  f620b1ae408d1184   

          date  solar_radiation_index  relative_humidity_near_surface  \
0     20241209                    NaN                              88   
1     20241209                    NaN                              88   
2     20241209                    NaN                              88   
3     20241209                    NaN                              90   
4     20241209                    NaN                              91   
...        ...                    ...                             ...   
1346  20241209                    NaN                              99   
1347  20241209                    NaN                              99   
1348  20241209                    NaN                              99   
1349  20241209                    NaN                              99   
1350  20241209                    NaN                              99   

      air_temperature_near_surface  dew_point_temperature  humidity_index  \
0                           281.05                 279.25          280.65   
1                           281.25                 279.45          280.85   
2                           281.15                 279.45          280.75   
3                           280.85                 279.45          280.45   
4                           280.75                 279.45          280.35   
...                            ...                    ...             ...   
1346                        279.25                 279.25          278.75   
1347                        279.25                 279.25          278.75   
1348                        279.25                 279.25          278.75   
1349                        279.25                 279.25          278.75   
1350                        279.25                 279.25          278.75   

      potential_temperature  altitude       lon        lat  \
0                     279.1       8.0  5.652387  51.982084   
1                     279.3       7.0  5.652359  51.982092   
2                     279.2       7.0  5.652255  51.982084   
3                     278.9       7.0  5.652168  51.982111   
4                     278.8       7.0  5.652277  51.982157   
...                     ...       ...       ...        ...   
1346                  277.3       3.0  5.666358  51.987296   
1

In [10]:
import geopandas as gpd
geo_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs=4326)
geo_df.explore(column = "altitude")

## Obtain Acronet data and other streams

For the meteotracker data, each track is stored in a single data file. For continuous streams of data like the acronet stations, it is stored in 5 minute granules.

In [11]:
[station["name"] for station in by_platform["acronet"]][:10] + [f"... + {len(by_platform["acronet"])} more"]

['Monte Santa Croce',
 'Piazza IV Novembre',
 'Villa Figoli',
 'Via Canavelle',
 'Trebiano',
 'Borgata Ponte',
 'San Genesio',
 'Rifugio Argentea',
 'Castiglione Chiavarese - Palestra',
 'Campo Sportivo Bajardo',
 '... + 50 more']

In [12]:
example_station = by_platform["acronet"][-1]
example_station

{'name': 'Arenella',
 'description': 'An Acronet station',
 'platform': 'acronet',
 'external_id': 'arenella',
 'internal_id': 'a7d22f057bb346b9',
 'location': [9.532785, 44.226414],
 'time_span': ['2024-12-08T19:25:00Z', '2024-12-08T19:30:00Z'],
 'authors': [{'name': 'acronet'}],
 'mars_request': {'class': 'rd',
  'expver': 'xxxx',
  'stream': 'lwda',
  'aggregation_type': 'chunked',
  'date': '20241208',
  'platform': 'acronet',
  'internal_id': 'a7d22f057bb346b9'}}

Giving the "mars_request" to the `list` endpoint gives the list of data granules available from the list endpoint. For the acronet data and other continuous streams, an additional key is needed "start_time". 

In [13]:
data_granules = session.get(url + "list", params = example_station["mars_request"])
sorted([g["mars_request"]["start_time"] for g in data_granules.json()])

['1925']

The `retrieve` endpoint simply concatenates the datafiles matched by a given mars request.

In [14]:
args = {
    "format" : "csv"
}

data = session.get(url + "retrieve", params = example_station["mars_request"] | args)

df = pd.read_csv(BytesIO(data.content))
df

class expver stream   project platform aggregation_type source_name  \
0    rd   xxxx   lwda  I-CHANGE  Acronet          chunked     Acronet   
1    rd   xxxx   lwda  I-CHANGE  Acronet          chunked     Acronet   
2    rd   xxxx   lwda  I-CHANGE  Acronet          chunked     Acronet   
3    rd   xxxx   lwda  I-CHANGE  Acronet          chunked     Acronet   
4    rd   xxxx   lwda  I-CHANGE  Acronet          chunked     Acronet   
5    rd   xxxx   lwda  I-CHANGE  Acronet          chunked     Acronet   

  external_id       internal_id      date  ...  wind_gust_direction  \
0    arenella  a7d22f057bb346b9  20241208  ...                  NaN   
1    arenella  a7d22f057bb346b9  20241208  ...                  NaN   
2    arenella  a7d22f057bb346b9  20241208  ...                  NaN   
3    arenella  a7d22f057bb346b9  20241208  ...                  NaN   
4    arenella  a7d22f057bb346b9  20241208  ...                  NaN   
5    arenella  a7d22f057bb346b9  20241208  ...                  NaN   

   air_temperature_near_surface  signal_strength  \
0                        281.45              NaN   
1                        281.45             23.0   
2                        281.35              NaN   
3                        281.35              NaN   
4                        281.35              NaN   
5                        281.35              NaN   

  relative_humidity_near_surface    altitude  wind_direction_near_surface  \
0                      63.900000  249.456701                    119.90000   
1                      64.000000  248.443170                    291.80002   
2                      64.200005  249.794566                    212.50000   
3                      64.200005  249.456701                    185.00000   
4                      64.000000  249.287772                     79.60000   
5                      64.000000  250.892705                     45.40000   

   wind_speed_near_surface  wind_gust  air_pressure_near_surface  \
0                 0.463000   1.080332                      99311   
1                 0.514444   0.823110                      99323   
2                 0.463000   1.183221                      99307   
3                 0.771666   1.286110                      99311   
4                 0.925999   1.697665                      99313   
5                 0.565888   1.749110                      99294   

   battery_level  
0      12.320000  
1      12.330000  
2      12.349999  
3      12.340000  
4      12.330000  
5      12.349999  

[6 rows x 26 columns]